In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target

print(X,y)
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


[[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]] [4.526 3.585 3.521 ... 0.923 0.847 0.894]


In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

class HousingData(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.X.shape[0]

# Instantiate training and test data
train_data = HousingData(X_train, y_train)
test_data = HousingData(X_test, y_test)

# Create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)


In [3]:
from torch import nn
from torch import optim

input_dim = X_train.shape[1]
hidden_dim = 10
output_dim = 1

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(in_features=input_dim, out_features=hidden_dim)
        self.layer_2 = nn.Linear(in_features=hidden_dim, out_features=output_dim)

    def forward(self, x):
        x = self.layer_1(x)
        x = nn.functional.relu(x)
        x = self.layer_2(x)
        return x



model = NeuralNetwork(input_dim, hidden_dim, output_dim)

learning_rate = 0.01
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


In [8]:
num_epochs = 100
from statistics import mean
loss_values = []

model.train()
for epoch in range(num_epochs):
    for X, y in train_dataloader:
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(-1))
        loss_values.append(loss.item())
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch+1} | Loss: {mean(loss_values)}")

print("Training Complete")


Epoch: 1 | Loss: 0.35698278329169103
Epoch: 2 | Loss: 0.3558821883485761
Epoch: 3 | Loss: 0.35516296352141896
Epoch: 4 | Loss: 0.3542248264041751
Epoch: 5 | Loss: 0.35503104649079864
Epoch: 6 | Loss: 0.35473911693314863
Epoch: 7 | Loss: 0.35477479416897656
Epoch: 8 | Loss: 0.35446337451291987
Epoch: 9 | Loss: 0.3542928032666881
Epoch: 10 | Loss: 0.35408748891060204
Epoch: 11 | Loss: 0.3539784643523411
Epoch: 12 | Loss: 0.353891329563076
Epoch: 13 | Loss: 0.35377255742108815
Epoch: 14 | Loss: 0.35376997850010233
Epoch: 15 | Loss: 0.35370637312223435
Epoch: 16 | Loss: 0.353626261846083
Epoch: 17 | Loss: 0.35424806995549213
Epoch: 18 | Loss: 0.35409285952268904
Epoch: 19 | Loss: 0.35407632525656135
Epoch: 20 | Loss: 0.3547125921158712
Epoch: 21 | Loss: 0.35482447391706623
Epoch: 22 | Loss: 0.35471774345504054
Epoch: 23 | Loss: 0.3545716205077578
Epoch: 24 | Loss: 0.35437224343544216
Epoch: 25 | Loss: 0.3543342321903207
Epoch: 26 | Loss: 0.3542007487889248
Epoch: 27 | Loss: 0.3540454030009

In [6]:
with torch.no_grad():
    total_loss = 0
    for X, y in test_dataloader:
        outputs = model(X)
        loss = loss_fn(outputs, y.unsqueeze(-1))
        total_loss += loss.item()

    print(f"Test Loss: {total_loss / len(test_dataloader)}")


Test Loss: 0.3644461122842935
